In [1]:
# PySpark API:
#http://spark.apache.org/docs/latest/api/python/index.html

import os
import sys
 
os.environ["SPARK_HOME"] = "/usr/spark2.4.3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")


# import and define spark context
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("appName")
sc = SparkContext(conf=conf)

# initialise sc
#from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, HiveContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

# test that config loaded correctly
rdd = sc.textFile("/data/mr/wordcount/input/")
print(rdd.take(10))
print(sc.version)

['The Project Gutenberg EBook of The Adventures of Sherlock Holmes', 'by Sir Arthur Conan Doyle', '(#15 in our series by Sir Arthur Conan Doyle)', '', 'Copyright laws are changing all over the world. Be sure to check the', 'copyright laws for your country before downloading or redistributing', 'this or any other Project Gutenberg eBook.', '', 'This header should be the first thing seen when viewing this Project', 'Gutenberg file.  Please do not remove it.  Do not change or edit the']
2.4.3


In [2]:
# Import libraries and other functions
from io import StringIO
from collections import namedtuple
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

import csv
import matplotlib.pyplot as plt
import gmplot
import pandas as pd

In [3]:
#import Accidental Drug Related Death using RDD
AccDrugDeath = sc.textFile("hdfs:///user/imat5322_user238740/Accidental_Drug_Related_Deaths_2012-2018 clean.csv")

In [4]:
#to see the first row of our data
AccDrugDeath.first()

'ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,DeathCounty,Location,LocationifOther,DescriptionofInjury,InjuryPlace,InjuryCity,InjuryCounty,InjuryState,COD,OtherSignifican,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo'

In [5]:
#Count the number of observations in the data
AccDrugDeath.count()

5106

In [6]:
#Take Ordered shows the data in hierachy
#Observe the data starts from 2012
AccDrugDeath.takeOrdered(10)

['12-0001,01/01/2012,DateofDeath,35,Male,White,HEBRON,TOLLAND,,HEBRON,TOLLAND,Residence,,Drug Use,Residence,,,,Cocaine and Heroin Toxicity,,Y,Y,,,,,,,,,,,,,,,,Accident,41.658069 -72.366324,41.658069 -72.366324,41.575155 -72.738288',
 '12-0002,03/01/2012,DateofDeath,41,Male,White,BRISTOL,HARTFORD,,BRISTOL,HARTFORD,Hospital,,Ingestion,Residence,,,,Multiple Drug Toxicity,,Y,,,,,,,,Y,,,,,,,,,Accident,41.673037 -72.945791,41.673037 -72.945791,41.575155 -72.738288',
 '12-0003,04/01/2012,DateofDeath,61,Male,Black,DANBURY,FAIRFIELD,,DANBURY,FAIRFIELD,Hospital,,Used Cocaine,Roadway,,,,Cocaine Toxicity,,,Y,,,,,,,,,,,,,,,,Accident,41.393666 -73.451539,41.393666 -73.451539,41.575155 -72.738288',
 "12-0004,05/01/2012,DateofDeath,51,Male,White,STRATFORD,FAIRFIELD,,BRIDGEPORT,FAIRFIELD,Other,Mother's Home,Ingestion,Residence,,,,Methadone Toxicity,,,,,,,,,,,Y,,,,,,,,Accident,41.179195 -73.189476,41.200888 -73.131323,41.575155 -72.738288",
 '12-0005,07/01/2012,DateofDeath,45,Male,White,HARTFORD,HARTFOR

In [7]:
AccDrugDeath.top(10)

['ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,DeathCounty,Location,LocationifOther,DescriptionofInjury,InjuryPlace,InjuryCity,InjuryCounty,InjuryState,COD,OtherSignifican,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo',
 '18-1018,31/12/2018,DateofDeath,57,Male,White,NEW HAVEN,NEW HAVEN,CT,NEW HAVEN,NEW HAVEN,Residence,,Substance Abuse,Residence,NEW HAVEN,NEW HAVEN,,Acute Intoxication due to the Combined Effects of Fentanyl Acetyl Fentanyl Heroin Cocaine and Methorphan,,Y,Y,Y,Y,,,,,,,,,,,,,Y,Accident,41.308252 -72.924161,41.308252 -72.924161,41.308252 -72.924161',
 '18-1017,30/12/2018,DateofDeath,51,Female,Black,WINDSOR,HARTFORD,CT,WINDSOR,HARTFORD,Residence,,Substance Abuse,Residence,WINDSOR,HARTFORD,,Acute Intoxication by the Combined Effects of Fentanyl H

In [8]:
#Find zero length records
AccDrugDeathMissing = AccDrugDeath.filter(lambda x: len(x) == 0)
AccDrugDeathMissing.count()

0

In [9]:
#Filter all zero length records
AccDrugDeathFlat = AccDrugDeath.filter(lambda x: len(x) < 0)
AccDrugDeathFlat.count()

0

In [10]:
AccDrugDeath2 = AccDrugDeath.map(lambda x: x.split(","))

In [11]:
AccDrugDeath2.take(2)

[['ID',
  'Date',
  'DateType',
  'Age',
  'Sex',
  'Race',
  'ResidenceCity',
  'ResidenceCounty',
  'ResidenceState',
  'DeathCity',
  'DeathCounty',
  'Location',
  'LocationifOther',
  'DescriptionofInjury',
  'InjuryPlace',
  'InjuryCity',
  'InjuryCounty',
  'InjuryState',
  'COD',
  'OtherSignifican',
  'Heroin',
  'Cocaine',
  'Fentanyl',
  'FentanylAnalogue',
  'Oxycodone',
  'Oxymorphone',
  'Ethanol',
  'Hydrocodone',
  'Benzodiazepine',
  'Methadone',
  'Amphet',
  'Tramad',
  'Morphine_NotHeroin',
  'Hydromorphone',
  'Other',
  'OpiateNOS',
  'AnyOpioid',
  'MannerofDeath',
  'DeathCityGeo',
  'ResidenceCityGeo',
  'InjuryCityGeo'],
 ['18-0478',
  '21/06/2018',
  'DateReported',
  '61',
  'Male',
  'White',
  'BRISTOL',
  'HARTFORD',
  'CT',
  'BRISTOL',
  'HARTFORD',
  'Residence',
  '',
  'Substance Abuse',
  'Residence',
  'BRISTOL',
  'HARTFORD',
  '',
  'Acute Intoxication From the Combined Effects of Fentanyl MethadoneHydroxyzine Olanzapine and Escitalopram',
  '',


In [12]:
#Explicitly create schema for all fields
schema_AccDrugDeath = StructType([ \
    StructField('ID', StringType(), True), \
    StructField('Date', StringType(), True), \
    StructField('DateType', StringType(), True), \
    StructField('Age', StringType(), True), \
    StructField('Sex', StringType(), True), \
    StructField('Race', StringType(), True), \
    StructField('ResidenceCity', StringType(), True), \
    StructField('ResidenceCounty', StringType(), True), \
    StructField('ResidenceState', StringType(), True), \
    StructField('DeathCity', StringType(), True), \
    StructField('DeathCounty', StringType(), True), \
    StructField('Location', StringType(), True), \
    StructField('LocationifOther', StringType(), True), \
    StructField('DescriptionofInjury', StringType(), True), \
    StructField('InjuryPlace', StringType(), True), \
    StructField('InjuryCity', StringType(), True), \
    StructField('InjuryCounty', StringType(), True), \
    StructField('InjuryState', StringType(), True), \
    StructField('COD', StringType(), True), \
    StructField('OtherSignifican', StringType(), True), \
    StructField('Heroin', StringType(), True), \
    StructField('Cocaine', StringType(), True), \
    StructField('Fentanyl', StringType(), True), \
    StructField('FentanylAnalogue', StringType(), True), \
    StructField('Oxycodone', StringType(), True), \
    StructField('Oxymorphone', StringType(), True), \
    StructField('Ethanol', StringType(), True), \
    StructField('Hydrocodone',StringType(), True), \
    StructField('Benzodiazepine', StringType(), True), \
    StructField('Methadone', StringType(), True), \
    StructField('Amphet', StringType(), True), \
    StructField('Tramad', StringType(), True), \
    StructField('Morphine_NotHeroin', StringType(), True), \
    StructField('Hydromorphone', StringType(), True), \
    StructField('Other', StringType(), True), \
    StructField('OpiateNOS', StringType(), True), \
    StructField('AnyOpioid', StringType(), True), \
    StructField('MannerofDeath', StringType(), True), \
    StructField('DeathCityGeo', StringType(), True), \
    StructField('ResidenceCityGeo', StringType(), True), \
    StructField('InjuryCityGeo', StringType(), True) ])
    


In [13]:
RDD_AccDrugDeath= AccDrugDeath2.map(lambda x: \
                   ((str(x[0]), str(x[1]), str(x[2]), \
                     str(x[3]), str(x[4]), str(x[5]), \
                     str(x[6]), str(x[7]), str(x[8]), \
                     str(x[9]), str(x[10]), str(x[11]), \
                     str(x[12]), str(x[13]), str(x[14]), \
                     str(x[15]), str(x[16]), str(x[17]), \
                     str(x[18]), str(x[19]), str(x[20]), \
                     str(x[21]), str(x[22]), str(x[23]), \
                     str(x[24]), str(x[25]), str(x[26]), \
                     str(x[27]), str(x[28]), str(x[29]), \
                     str(x[30]), str(x[31]), str(x[32]), \
                     str(x[33]), str(x[34]), str(x[35]), \
                     str(x[36]), str(x[37]), str(x[38]), \
                     str(x[39]), str(x[40]) )))

In [14]:
RDD_AccDrugDeath.take(2)

[('ID',
  'Date',
  'DateType',
  'Age',
  'Sex',
  'Race',
  'ResidenceCity',
  'ResidenceCounty',
  'ResidenceState',
  'DeathCity',
  'DeathCounty',
  'Location',
  'LocationifOther',
  'DescriptionofInjury',
  'InjuryPlace',
  'InjuryCity',
  'InjuryCounty',
  'InjuryState',
  'COD',
  'OtherSignifican',
  'Heroin',
  'Cocaine',
  'Fentanyl',
  'FentanylAnalogue',
  'Oxycodone',
  'Oxymorphone',
  'Ethanol',
  'Hydrocodone',
  'Benzodiazepine',
  'Methadone',
  'Amphet',
  'Tramad',
  'Morphine_NotHeroin',
  'Hydromorphone',
  'Other',
  'OpiateNOS',
  'AnyOpioid',
  'MannerofDeath',
  'DeathCityGeo',
  'ResidenceCityGeo',
  'InjuryCityGeo'),
 ('18-0478',
  '21/06/2018',
  'DateReported',
  '61',
  'Male',
  'White',
  'BRISTOL',
  'HARTFORD',
  'CT',
  'BRISTOL',
  'HARTFORD',
  'Residence',
  '',
  'Substance Abuse',
  'Residence',
  'BRISTOL',
  'HARTFORD',
  '',
  'Acute Intoxication From the Combined Effects of Fentanyl MethadoneHydroxyzine Olanzapine and Escitalopram',
  '',


In [15]:
DF_AccDrugDeath = sqlContext.createDataFrame(RDD_AccDrugDeath, schema_AccDrugDeath)

DF_AccDrugDeath.take(2)

[Row(ID='ID', Date='Date', DateType='DateType', Age='Age', Sex='Sex', Race='Race', ResidenceCity='ResidenceCity', ResidenceCounty='ResidenceCounty', ResidenceState='ResidenceState', DeathCity='DeathCity', DeathCounty='DeathCounty', Location='Location', LocationifOther='LocationifOther', DescriptionofInjury='DescriptionofInjury', InjuryPlace='InjuryPlace', InjuryCity='InjuryCity', InjuryCounty='InjuryCounty', InjuryState='InjuryState', COD='COD', OtherSignifican='OtherSignifican', Heroin='Heroin', Cocaine='Cocaine', Fentanyl='Fentanyl', FentanylAnalogue='FentanylAnalogue', Oxycodone='Oxycodone', Oxymorphone='Oxymorphone', Ethanol='Ethanol', Hydrocodone='Hydrocodone', Benzodiazepine='Benzodiazepine', Methadone='Methadone', Amphet='Amphet', Tramad='Tramad', Morphine_NotHeroin='Morphine_NotHeroin', Hydromorphone='Hydromorphone', Other='Other', OpiateNOS='OpiateNOS', AnyOpioid='AnyOpioid', MannerofDeath='MannerofDeath', DeathCityGeo='DeathCityGeo', ResidenceCityGeo='ResidenceCityGeo', Injur

In [16]:
DF_AccDrugDeath.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- DateType: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- ResidenceCity: string (nullable = true)
 |-- ResidenceCounty: string (nullable = true)
 |-- ResidenceState: string (nullable = true)
 |-- DeathCity: string (nullable = true)
 |-- DeathCounty: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- LocationifOther: string (nullable = true)
 |-- DescriptionofInjury: string (nullable = true)
 |-- InjuryPlace: string (nullable = true)
 |-- InjuryCity: string (nullable = true)
 |-- InjuryCounty: string (nullable = true)
 |-- InjuryState: string (nullable = true)
 |-- COD: string (nullable = true)
 |-- OtherSignifican: string (nullable = true)
 |-- Heroin: string (nullable = true)
 |-- Cocaine: string (nullable = true)
 |-- Fentanyl: string (nullable = true)
 |-- FentanylAnalogue: string (nullable = true)


In [17]:
#Spilt DeathCityGeo into seperate columns of Latitude and Longitude
split_AccDrugDeath = DF_AccDrugDeath.withColumn('DeathCityLong', split(DF_AccDrugDeath['DeathCityGeo'], ' ')[1])
split_AccDrugDeath.take(2)


[Row(ID='ID', Date='Date', DateType='DateType', Age='Age', Sex='Sex', Race='Race', ResidenceCity='ResidenceCity', ResidenceCounty='ResidenceCounty', ResidenceState='ResidenceState', DeathCity='DeathCity', DeathCounty='DeathCounty', Location='Location', LocationifOther='LocationifOther', DescriptionofInjury='DescriptionofInjury', InjuryPlace='InjuryPlace', InjuryCity='InjuryCity', InjuryCounty='InjuryCounty', InjuryState='InjuryState', COD='COD', OtherSignifican='OtherSignifican', Heroin='Heroin', Cocaine='Cocaine', Fentanyl='Fentanyl', FentanylAnalogue='FentanylAnalogue', Oxycodone='Oxycodone', Oxymorphone='Oxymorphone', Ethanol='Ethanol', Hydrocodone='Hydrocodone', Benzodiazepine='Benzodiazepine', Methadone='Methadone', Amphet='Amphet', Tramad='Tramad', Morphine_NotHeroin='Morphine_NotHeroin', Hydromorphone='Hydromorphone', Other='Other', OpiateNOS='OpiateNOS', AnyOpioid='AnyOpioid', MannerofDeath='MannerofDeath', DeathCityGeo='DeathCityGeo', ResidenceCityGeo='ResidenceCityGeo', Injur

In [18]:
##Split ResidenceCityGeo column into Latitude and Longitiude
split_AccDrugResidence= split_AccDrugDeath.withColumn('ResidenceCityLong', split(split_AccDrugDeath['ResidenceCityGeo'], ' ')[1])
split_AccDrugResidence.take(2)

[Row(ID='ID', Date='Date', DateType='DateType', Age='Age', Sex='Sex', Race='Race', ResidenceCity='ResidenceCity', ResidenceCounty='ResidenceCounty', ResidenceState='ResidenceState', DeathCity='DeathCity', DeathCounty='DeathCounty', Location='Location', LocationifOther='LocationifOther', DescriptionofInjury='DescriptionofInjury', InjuryPlace='InjuryPlace', InjuryCity='InjuryCity', InjuryCounty='InjuryCounty', InjuryState='InjuryState', COD='COD', OtherSignifican='OtherSignifican', Heroin='Heroin', Cocaine='Cocaine', Fentanyl='Fentanyl', FentanylAnalogue='FentanylAnalogue', Oxycodone='Oxycodone', Oxymorphone='Oxymorphone', Ethanol='Ethanol', Hydrocodone='Hydrocodone', Benzodiazepine='Benzodiazepine', Methadone='Methadone', Amphet='Amphet', Tramad='Tramad', Morphine_NotHeroin='Morphine_NotHeroin', Hydromorphone='Hydromorphone', Other='Other', OpiateNOS='OpiateNOS', AnyOpioid='AnyOpioid', MannerofDeath='MannerofDeath', DeathCityGeo='DeathCityGeo', ResidenceCityGeo='ResidenceCityGeo', Injur

In [19]:

AccDrugDeathNew = split_AccDrugResidence.drop('DescriptionofInjury', 'InjuryPlace', 'InjuryCity', 'InjuryCounty', 'InjuryState', 'COD', 'OtherSignifican', 'InjuryCityGeo') 
AccDrugDeathNew.take(2)

[Row(ID='ID', Date='Date', DateType='DateType', Age='Age', Sex='Sex', Race='Race', ResidenceCity='ResidenceCity', ResidenceCounty='ResidenceCounty', ResidenceState='ResidenceState', DeathCity='DeathCity', DeathCounty='DeathCounty', Location='Location', LocationifOther='LocationifOther', Heroin='Heroin', Cocaine='Cocaine', Fentanyl='Fentanyl', FentanylAnalogue='FentanylAnalogue', Oxycodone='Oxycodone', Oxymorphone='Oxymorphone', Ethanol='Ethanol', Hydrocodone='Hydrocodone', Benzodiazepine='Benzodiazepine', Methadone='Methadone', Amphet='Amphet', Tramad='Tramad', Morphine_NotHeroin='Morphine_NotHeroin', Hydromorphone='Hydromorphone', Other='Other', OpiateNOS='OpiateNOS', AnyOpioid='AnyOpioid', MannerofDeath='MannerofDeath', DeathCityGeo='DeathCityGeo', ResidenceCityGeo='ResidenceCityGeo', DeathCityLong=None, ResidenceCityLong=None),
 Row(ID='18-0478', Date='21/06/2018', DateType='DateReported', Age='61', Sex='Male', Race='White', ResidenceCity='BRISTOL', ResidenceCounty='HARTFORD', Resid

In [20]:
AccDrugDeathGeo=AccDrugDeathNew.withColumn("DeathCityGeo",AccDrugDeathNew["DeathCityGeo"].cast(FloatType()))
AccDrugDeathGeo

DataFrame[ID: string, Date: string, DateType: string, Age: string, Sex: string, Race: string, ResidenceCity: string, ResidenceCounty: string, ResidenceState: string, DeathCity: string, DeathCounty: string, Location: string, LocationifOther: string, Heroin: string, Cocaine: string, Fentanyl: string, FentanylAnalogue: string, Oxycodone: string, Oxymorphone: string, Ethanol: string, Hydrocodone: string, Benzodiazepine: string, Methadone: string, Amphet: string, Tramad: string, Morphine_NotHeroin: string, Hydromorphone: string, Other: string, OpiateNOS: string, AnyOpioid: string, MannerofDeath: string, DeathCityGeo: float, ResidenceCityGeo: string, DeathCityLong: string, ResidenceCityLong: string]

In [21]:
AccDrugDeathLong=AccDrugDeathGeo.withColumn("DeathCityLong",AccDrugDeathGeo["DeathCityLong"].cast(FloatType()))
AccDrugDeathLong

DataFrame[ID: string, Date: string, DateType: string, Age: string, Sex: string, Race: string, ResidenceCity: string, ResidenceCounty: string, ResidenceState: string, DeathCity: string, DeathCounty: string, Location: string, LocationifOther: string, Heroin: string, Cocaine: string, Fentanyl: string, FentanylAnalogue: string, Oxycodone: string, Oxymorphone: string, Ethanol: string, Hydrocodone: string, Benzodiazepine: string, Methadone: string, Amphet: string, Tramad: string, Morphine_NotHeroin: string, Hydromorphone: string, Other: string, OpiateNOS: string, AnyOpioid: string, MannerofDeath: string, DeathCityGeo: float, ResidenceCityGeo: string, DeathCityLong: float, ResidenceCityLong: string]

In [22]:
AccDrugResidenceGeo= AccDrugDeathLong.withColumn("ResidenceCityGeo", AccDrugDeathLong["ResidenceCityGeo"].cast(FloatType()))
AccDrugResidenceGeo

DataFrame[ID: string, Date: string, DateType: string, Age: string, Sex: string, Race: string, ResidenceCity: string, ResidenceCounty: string, ResidenceState: string, DeathCity: string, DeathCounty: string, Location: string, LocationifOther: string, Heroin: string, Cocaine: string, Fentanyl: string, FentanylAnalogue: string, Oxycodone: string, Oxymorphone: string, Ethanol: string, Hydrocodone: string, Benzodiazepine: string, Methadone: string, Amphet: string, Tramad: string, Morphine_NotHeroin: string, Hydromorphone: string, Other: string, OpiateNOS: string, AnyOpioid: string, MannerofDeath: string, DeathCityGeo: float, ResidenceCityGeo: float, DeathCityLong: float, ResidenceCityLong: string]

In [23]:
AccDrugResidenceLong= AccDrugResidenceGeo.withColumn("ResidenceCityLong", AccDrugResidenceGeo["ResidenceCityLong"].cast(FloatType()))
AccDrugResidenceLong

DataFrame[ID: string, Date: string, DateType: string, Age: string, Sex: string, Race: string, ResidenceCity: string, ResidenceCounty: string, ResidenceState: string, DeathCity: string, DeathCounty: string, Location: string, LocationifOther: string, Heroin: string, Cocaine: string, Fentanyl: string, FentanylAnalogue: string, Oxycodone: string, Oxymorphone: string, Ethanol: string, Hydrocodone: string, Benzodiazepine: string, Methadone: string, Amphet: string, Tramad: string, Morphine_NotHeroin: string, Hydromorphone: string, Other: string, OpiateNOS: string, AnyOpioid: string, MannerofDeath: string, DeathCityGeo: float, ResidenceCityGeo: float, DeathCityLong: float, ResidenceCityLong: float]

In [24]:
AccDrugDate= AccDrugResidenceLong.withColumn("Date", AccDrugResidenceLong["Date"].cast(DateType()))
AccDrugDate

DataFrame[ID: string, Date: date, DateType: string, Age: string, Sex: string, Race: string, ResidenceCity: string, ResidenceCounty: string, ResidenceState: string, DeathCity: string, DeathCounty: string, Location: string, LocationifOther: string, Heroin: string, Cocaine: string, Fentanyl: string, FentanylAnalogue: string, Oxycodone: string, Oxymorphone: string, Ethanol: string, Hydrocodone: string, Benzodiazepine: string, Methadone: string, Amphet: string, Tramad: string, Morphine_NotHeroin: string, Hydromorphone: string, Other: string, OpiateNOS: string, AnyOpioid: string, MannerofDeath: string, DeathCityGeo: float, ResidenceCityGeo: float, DeathCityLong: float, ResidenceCityLong: float]

In [25]:
AccDrugAge= AccDrugDate.withColumn("Age", AccDrugDate["Age"].cast(IntegerType()))
AccDrugAge

DataFrame[ID: string, Date: date, DateType: string, Age: int, Sex: string, Race: string, ResidenceCity: string, ResidenceCounty: string, ResidenceState: string, DeathCity: string, DeathCounty: string, Location: string, LocationifOther: string, Heroin: string, Cocaine: string, Fentanyl: string, FentanylAnalogue: string, Oxycodone: string, Oxymorphone: string, Ethanol: string, Hydrocodone: string, Benzodiazepine: string, Methadone: string, Amphet: string, Tramad: string, Morphine_NotHeroin: string, Hydromorphone: string, Other: string, OpiateNOS: string, AnyOpioid: string, MannerofDeath: string, DeathCityGeo: float, ResidenceCityGeo: float, DeathCityLong: float, ResidenceCityLong: float]

In [26]:
AccDrugAge.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- DateType: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- ResidenceCity: string (nullable = true)
 |-- ResidenceCounty: string (nullable = true)
 |-- ResidenceState: string (nullable = true)
 |-- DeathCity: string (nullable = true)
 |-- DeathCounty: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- LocationifOther: string (nullable = true)
 |-- Heroin: string (nullable = true)
 |-- Cocaine: string (nullable = true)
 |-- Fentanyl: string (nullable = true)
 |-- FentanylAnalogue: string (nullable = true)
 |-- Oxycodone: string (nullable = true)
 |-- Oxymorphone: string (nullable = true)
 |-- Ethanol: string (nullable = true)
 |-- Hydrocodone: string (nullable = true)
 |-- Benzodiazepine: string (nullable = true)
 |-- Methadone: string (nullable = true)
 |-- Amphet: string (nullable = true)
 |-- Tramad: st

In [27]:
sqlContext.registerDataFrameAsTable(AccDrugAge, "Accidental_Drug_Related_Deaths")

sqlContext.sql("SELECT * FROM Accidental_Drug_Related_Deaths").show()


+-------+----+------------+----+------+-----+-------------+---------------+--------------+-------------+-----------+---------+---------------+------+-------+--------+----------------+---------+-----------+-------+-----------+--------------+---------+------+------+------------------+-------------+-----+---------+---------+-------------+------------+----------------+-------------+-----------------+
|     ID|Date|    DateType| Age|   Sex| Race|ResidenceCity|ResidenceCounty|ResidenceState|    DeathCity|DeathCounty| Location|LocationifOther|Heroin|Cocaine|Fentanyl|FentanylAnalogue|Oxycodone|Oxymorphone|Ethanol|Hydrocodone|Benzodiazepine|Methadone|Amphet|Tramad|Morphine_NotHeroin|Hydromorphone|Other|OpiateNOS|AnyOpioid|MannerofDeath|DeathCityGeo|ResidenceCityGeo|DeathCityLong|ResidenceCityLong|
+-------+----+------------+----+------+-----+-------------+---------------+--------------+-------------+-----------+---------+---------------+------+-------+--------+----------------+---------+-------